In [163]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from tqdm import tqdm_notebook as tqdm


from librosa import display
import librosa
import numpy as np
from numpy import array

In [164]:
#forming a panda dataframe from the metadata file
csv_path = "UrbanSound8K/metadata/UrbanSound8K.csv"
data=pd.read_csv(csv_path)

In [167]:
#preprocessing using entire feature set

x_train=[]
x_test=[]
y_train=[]
y_test=[]
feature_size = 80
path="UrbanSound8K/audio/fold"
columns =["mfcc", "melspectrogram", "chroma_stft", "chroma_cq", "chroma_cens"]
data=data.astype(object)

def get_features(i):
    fold_no=str(data.iloc[i]["fold"])
    file=data.iloc[i]["slice_file_name"]
    label=data.iloc[i]["classID"]
    filename=path+fold_no+"/"+file
    y,sr=librosa.load(filename)
    
    features=np.reshape(np.vstack((
        np.mean(librosa.feature.mfcc(y, sr, n_mfcc=feature_size).T,axis=0),
        np.mean(librosa.feature.melspectrogram(y=y, sr=sr, n_mels=feature_size,fmax=8000).T,axis=0),
        np.mean(librosa.feature.chroma_stft(y=y, sr=sr,n_chroma=feature_size).T,axis=0),
        np.mean(librosa.feature.chroma_cqt(y=y, sr=sr,n_chroma=feature_size).T,axis=0),
        np.mean(librosa.feature.chroma_cens(y=y, sr=sr,n_chroma=feature_size).T,axis=0))
    ),(feature_size,5))
    
    
    data.at[i, "feature"] = np.array_repr(features.flatten()).replace('\n', '')

#I have no clue why, but you need to run this cell twice or else it'll only get ValueErrors
for i in tqdm(range(len(data))):
    try:
        get_features(i)
    except ValueError:
        print(i)
data[:10]

c:\users\anton\anaconda3\envs\ml-agents\lib\site-packages\librosa\core\pitch.py:146: UserWarning: Trying to estimate tuning from empty frequency set.
  warnings.warn('Trying to estimate tuning from empty frequency set.')


,slice_file_name,fsID,start,end,salience,fold,classID,class,feature
0,100032-3-0-0.wav,100032,0,0.317551,1,5,3,dog_bark,"array([-2.09935776e+02, 6.34226341e+01, -1.23..."
1,100263-2-0-117.wav,100263,58.5,62.5,1,5,2,children_playing,"array([-4.17291107e+02, 1.00148506e+02, -4.38..."
2,100263-2-0-121.wav,100263,60.5,64.5,1,5,2,children_playing,"array([-4.52774475e+02, 1.13243027e+02, -3.85..."
3,100263-2-0-126.wav,100263,63,67,1,5,2,children_playing,"array([-4.06865326e+02, 9.20699234e+01, -2.59..."
4,100263-2-0-137.wav,100263,68.5,72.5,1,5,2,children_playing,"array([-4.40051483e+02, 1.04752792e+02, -4.36..."
5,100263-2-0-143.wav,100263,71.5,75.5,1,5,2,children_playing,"array([-4.41531830e+02, 1.09718697e+02, -2.66..."
6,100263-2-0-161.wav,100263,80.5,84.5,1,5,2,children_playing,"array([-4.69740143e+02, 1.10137512e+02, -1.93..."
7,100263-2-0-3.wav,100263,1.5,5.5,1,5,2,children_playing,"array([-4.57554596e+02, 1.07447548e+02, -2.00..."
8,100263-2-0-36.wav,100263,18,22,1,5,2,children_playing,"array([-4.66591217e+02, 1.18987900e+02, -3.07..."
9,100648-1-0-0.wav,100648,4.8234,5.47193,2,10,1,car_horn,"array([-1.87959152e+02, 1.03357376e+02, -3.20..."


In [82]:
data.to_csv(csv_path,index=False, )

In [205]:
from keras import Sequential
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping,TensorBoard
from keras.layers import Dense,Conv1D,MaxPooling1D,Flatten,Dropout
from keras.utils.np_utils import to_categorical

In [213]:
#adding layers and forming the model
def get_model():
    model = Sequential()
    model.add(Conv1D(32,kernel_size=5,strides=1,padding="same",activation="relu",input_shape=(feature_size,5)))
    model.add(MaxPooling1D(padding="same"))
    
    model.add(Conv1D(64,kernel_size=5,strides=1,padding="same",activation="relu"))
    model.add(MaxPooling1D(padding="same"))
    model.add(Dropout(0.3))

    model.add(Conv1D(128,kernel_size=5,strides=1,padding="same",activation="relu"))

    model.add(MaxPooling1D(padding="same"))
    model.add(Dropout(0.3))

    model.add(Flatten())

    model.add(Dense(256,activation="relu"))
    model.add(Dropout(0.3))

    model.add(Dense(256,activation="relu"))
    model.add(Dropout(0.3))

    model.add(Dense(10,activation="softmax"))
    return model

def get_callbacks(name_weights, patience_lr):
     
    mcp_save = ModelCheckpoint("weights/{}.h5".format(name_weights), save_best_only=True, monitor='val_loss', mode='min')
    reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=patience_lr, verbose=0, epsilon=1e-4, mode='auto')
    tensorboard = TensorBoard(log_dir='logs/{}'.format(name_weights))
    es = EarlyStopping(monitor='val_acc', mode='max', min_delta=1, patience=30, verbose=0)
    return [mcp_save, reduce_lr_loss, tensorboard, es]

In [214]:
name_run= "to_save"

def train(j):
    train_df = data[(data.fold != j)]
    test_df = data[(data.fold == j)]

    x_train = np.asarray([eval(arr).reshape(80,5) for arr in train_df["feature"]])
    y_train = to_categorical(train_df["classID"].to_numpy(), num_classes=10)
    

    x_test = np.asarray([eval(arr).reshape(80,5) for arr in test_df["feature"]])
    y_test = to_categorical(test_df["classID"].to_numpy(), num_classes=10)
    

    name_check = "{}_fold_{}".format(name_run,str(j))

    model = get_model()

    callbacks = get_callbacks(name_check, patience_lr=10)
    model.compile(optimizer="adam",loss="categorical_crossentropy",metrics=["accuracy"])
    model.fit(x_train,y_train,batch_size=50,epochs=150,validation_data=(x_test,y_test),callbacks = callbacks,verbose=0)

# for j in range(1,11):
#     train(j)

train(10)

In [220]:
import os
import keras
import tensorflow as tf
import os.path as p
def save_model(model, model_name, version):
    MODEL_DIR = p.join("serving", model_name)
    export_path = os.path.join(MODEL_DIR, str(version))
    tf.saved_model.simple_save(
    keras.backend.get_session(),
    export_path,
    inputs={'input_image': model.input},
    outputs={t.name:t for t in model.outputs})
    print(export_path)

def load_model(fold_nr, name):
    model = get_model()
    model_name = "{}_fold_{}".format(name_run,str(fold_nr))

    model.load_weights(p.join("weights", "{}.h5".format(model_name)))
    return model

In [221]:
version = 1
model = load_model(10, name_run)
save_model(model, name_run, version)

INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: b'serving\\to_save\\1\\saved_model.pb'
serving\to_save\1
